In [1]:
import polars
import hvplot.polars

import holoviews as hv

from polars import col

HEIGHT = 600
WIDTH = 1200

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [5]:
amd = polars.read_csv("../tests/data/AMD.csv", try_parse_dates=True).tail(50)
amd.head()

Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Time Zone
date,f64,f64,f64,f64,i64,f64,f64,str
2024-05-29,167.460007,168.479996,162.910004,165.139999,56537500,0.0,0.0,"""EDT"""
2024-05-30,167.899994,168.75,163.800003,166.75,46479900,0.0,0.0,"""EDT"""
2024-05-31,166.649994,169.5,160.070007,166.899994,64331900,0.0,0.0,"""EDT"""
2024-06-03,170.820007,171.080002,160.910004,163.550003,59157600,0.0,0.0,"""EDT"""
2024-06-04,162.839996,164.830002,158.869995,159.990005,48157200,0.0,0.0,"""EDT"""


In [6]:
import indicatory.deviations as dev

In [7]:
sd = dev.standard_deviation(amd)
var = dev.variance(amd)
aad = dev.average_absolute_deviation(amd)

In [10]:
ohlc = amd.hvplot.ohlc(x="Date", y=["Open", "High", "Low", "Close"], grid=True, width=WIDTH, height=HEIGHT)
sd_plot = sd.hvplot.line(x="Date", y="STD Close", grid=True, width=WIDTH, height=int(HEIGHT / 3))
var_plot = var.hvplot.line(x="Date", y="VAR Close", grid=True, width=WIDTH, height=int(HEIGHT / 3))
aad_plot = aad.hvplot.line(x="Date", y="AAD", grid=True, width=WIDTH, height=int(HEIGHT / 3))

hv.Layout(ohlc + sd_plot + var_plot + aad_plot).cols(1)

:Layout
   .Overlay.I :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .Curve.I   :Curve   [Date]   (STD Close)
   .Curve.II  :Curve   [Date]   (VAR Close)
   .Curve.III :Curve   [Date]   (AAD)

In [11]:
sd_band = dev.standard_deviation_bands(amd)
aad_band = dev.average_absolute_deviation_bands(amd)

In [14]:
sd_band_plot = sd_band.hvplot.line(x="Date", y=["SD lower", "SD upper"], grid=True, width=WIDTH, height=HEIGHT)
aad_band_plot = aad_band.hvplot.line(x="Date", y=["AAD lower", "AAD upper"], grid=True, width=WIDTH, height=HEIGHT)

hv.Layout(ohlc * sd_band_plot + ohlc * aad_band_plot).cols(1)

:Layout
   .Overlay.I  :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
      .NdOverlay.I  :NdOverlay   [Variable]
         :Curve   [Date]   (value)
   .Overlay.II :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
      .NdOverlay.I  :NdOverlay   [Variable]
         :Curve   [Date]   (value)